### ESP8266 como access point


El modo AP permite crear una red wifi desde el esp8266. Para realizar el mismo se utilizan dos funciones 

- WiFi.mode(WIFI_AP)     configura el esp8266 para que funcione como access point 
- WiFi.softAP(…)Genera una red WiFi, que devuelve true si la red WiFi se ha creado correctamente y false en caso contrario.


Un programa simple que crea una red WIFI seria:

```
#include <ESP8266WiFi.h>        // Include the Wi-Fi library  
  
// Configuración de la WiFi generada  
const char *ssid = "ssid";  
const char *password = "password";  
  
void setup() {  
  Serial.begin(115200);  
  delay(10);  
    
  WiFi.mode(WIFI_AP);  
  while(!WiFi.softAP(ssid, password))  
  {  
   Serial.println(".");  
    delay(100);  
  }  
    
  Serial.print("Iniciado AP ");  
  Serial.println(ssid);  
  Serial.print("IP address:\t");  
  Serial.println(WiFi.softAPIP());  
  
}  

void loop() { }

```

#### ESP8266 como access point y webserver

Este programa permite crear una red wifi desde el ESP8266 y que tambien actue como webserver. Utiliza los siguientes archivos.
- config.h
- wifi.hpp
- server.hpp
- archivo principal

##### config.h

En este arcchivo se presentA una pagina web que controla un led.

```
//---------------------------------------------------------------
// Texto en HTML

const char MAIN_page[] PROGMEM = R"=====(  
<!DOCTYPE html>  
<html>  
<body>  
<center>  
<h1>Programa para prender/apagar un LED </h1><br>
Apretar para <a href="ledP">Prender LED</a><br>
Apretar para <a href="ledA">Apagar LED</a><br>
<hr>
</center>

</body>
</html>
)=====";

const char* ssid="######";
const char* password="******";
 
```

#### wifi.hpp
Se configura el esp8266 como access point, se utiliza para generar una red wifi con el esp8266

```
//Declarar un objeto de la clase ESP8266WebServer  

ESP8266WebServer server(80); //Server on port 80  

// Inicializa la red wifi como access point

inicializaWifi_AP()
{
 WiFi.mode(WIFI_AP);
   while(!WiFi.softAP("labelec","nada"))
  {
   Serial.print(".");
    delay(100);
  }

  IPAddress myIP = WiFi.softAPIP();
  Serial.print("AP IP address: ");
  Serial.println(myIP); 
  Serial.println("");  
}
```

#### server.hpp

En la libreria se encuentran las funciones que realizan la vinculacion entre la pagina web y los actuadores y sensores
las funciones que se encuentran son:

- controlRaiz: Carga la pagina cuando alguna persona intenta conectarse a la misma
- prender: Enciende el led de la placa
- apagar: apaga el led de la placa
- iniciarServidor: Funcion que ante el pedido que se realiza por la pagina Web distribuye las ordenes en las distintas funciones


```
// inicializa el servidor web

void controRaiz() {  
 Serial.println("Pagina original ");  
 String s = MAIN_page; //Read HTML contents  
 server.send(200, "text/html", s); //Send web page  
}  

void prender() {   
 Serial.println("LED prendido");  
 digitalWrite(LED,LOW); //LED is connected in reverse  
 server.send(200, "text/html", "El LED esta prendido");  
}  

void apagar() {   
 Serial.println("LED apagado");  
 digitalWrite(LED,HIGH); //LED off  
 server.send(200, "text/html", "El LED esta apagado");   
}  


iniciarServidor() {

server.on("/", controlRaiz);
server.on("/Prender", prender); //Rutina mueve el brazo
server.on("/Apagar", apagar); //Rutina mueve el brazo
server.begin();  
}


```

#### Archivo principal

```
#include <ESP8266WiFi.h>  
#include <WiFiClient.h>  
#include <ESP8266WebServer.h> 
#include "config.h"
#include "WIFI.hpp"
#include "server.hpp"

//==============================================================  
//                  SETUP  
//==============================================================  
void setup(void){  
  Serial.begin(115200);  
  inicializaWifi_AP();
  inicializaServidorWeb();

  //Configuro el pin LED como salida  
  pinMode(LED,OUTPUT);   
  //Apaga el LED  
  digitalWrite(LED,HIGH);  
  // Carga las rutinas que se utilizaran por el servidor WEB
 
  Serial.println("HTTP server started");  
}  
//==============================================================  
//                     LOOP  
//==============================================================  
void loop(void){  
  server.handleClient();          //Handle clien  t requests
}
```

#### Servomotor controlado por pagina WEB usando ESP8266 

Se utiliza el circuito de la figura
<img src="img/nodemcu_servo.png" alt="Imagen"></img>
El programa tiene los siguientes archivos:

- config.h
- actuador.hpp
- WIFI.hpp
- server.hpp
- archivo principal


##### config.h

```
const char Mi_pagina[] PROGMEM = R"=====(
<!DOCTYPE html>  <html> 
<head><meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0, user-scalable=no\">
<title>CONTROL DE servomotor</title>
</head>

<body>  
<h1>Servomotor </h1><br>

<form method="post" action="/angulo">
Angulo:
<input type="text" name="angul" size="20">
<br>
<input type="submit" value="enviar">
</form>

</body>
</html>
)=====";

const char* ssid="########";
const char* password="********";
```

##### actuador.hpp

Crea el objeto servo y la funcion iniciarServo que configura el servomotor

```
Servo uno;

void iniciarServo()
{
  pinMode(D1, OUTPUT);
  uno.attach(D1);
}
```

In [ ]:
#### wifi.hpp

Configura y conecta la red wifi


```
void configuracion_wifi_STA()
{
  WiFi.begin(ssid, password);
  Serial.println(" ");
  Serial.println("Conectando");
  while (WiFi.status() != WL_CONNECTED)
  {
    delay(500);
    Serial.print(".");
  }
  Serial.println();
  Serial.print("Conectado, dirección IP: ");
  Serial.println(WiFi.localIP());
}

void configuracion_wifi_AP()
{
  WiFi.mode(WIFI_AP);  
  while(!WiFi.softAP(ssid, password))  
  {  
   Serial.println(".");  
    delay(100);  
  }  
  Serial.print("Iniciado AP ");  
  Serial.println(ssid);  
  Serial.print("IP address:\t");  
  Serial.println(WiFi.softAPIP());  
}
```


In [ ]:
#### server.hpp

Configura, inicia y controla el servidor Web, las funciones son:
- controlRaiz: Carga la pagina principal
- controlAngulo Controla el servomotor desde la pagina
- iniciarServidor:  Contorla la pagina con sus distintas funciones


```
ESP8266WebServer server(80); //Server on port 80

void controlRaiz() {
  int i;
  Serial.println("Pagina original ");
  String s = Mi_pagina; //Read HTML contents
  server.send(200, "text/html", s); //Send web page
}

void controlAngulo() {
  String a;
  int valor;
  a=server.arg("angul");
  valor=a.toInt();
  uno.write(valor);
  Serial.println(server.arg("angul"));
  Serial.println(valor);
  String s = Mi_pagina; //Read HTML contents
  delay(200);
  server.send(200, "text/html", s); //Send web page
}
void iniciarServidor()
{
  server.on("/", controlRaiz);
  server.on("/angulo", controlAngulo);
  server.begin();
}
```

#### Archivo principal

Carga todas las librerias e inicializa el dispositivo, inicializa wifi y el servidorweb

```
void setup()  
{   
Serial.begin(115200);  
pinMode(LED_BUILTIN,OUTPUT);
Serial.println("Hola");
iniciarServo();  
configuracion_wifi_STA();  
iniciar_servidor();
}  

void loop() {
  server.handleClient();
  }
```

### Medidor de distancia desde pagina WEB

Se utiliza un esp8266  como WEB server y un sensor HC-sr04 basado en ultrasonido para medir distancia y conectarlo a la red. El circuito se muestra en la figura.

<img src="img/nodemcu_ultrasonido_bb.jpg" alt="Imagen"></img>

El programa tiene los siguientes archivos:

- config.h
- sensor.hpp
- WIFI.hpp
- server.hpp
- archivo principal



#### config.h

En este archivo se encuentra una parte de la pagina WEB y las claves para ingresar a la red WIFI



```
const char Mi_pagina[] PROGMEM = R"=====(
<!DOCTYPE html>  <html> 
<head><meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0, user-scalable=no\">
<title>Medidor de distancia</title>
</head>

<body>  
<h1>Medidor de distancia </h1><br>

)=====";


String s2=" </body> </html>";

const char* ssid="######";
const char* password="******";


#### sensor.hpp

Archivo donde se encuentran las funciones que permiten utilizar y controlar el medidor de distancia. Las funciones son:
 - IniciarUltrasonido(): Inicializa el sensor
 - medirDistancia(): mide la distancia de un objeto al sensor

```
long distancia;  
long tiempo;  

void iniciarUltrasonido() {
pinMode(D6,OUTPUT);  /* Trigger */
pinMode(D5,INPUT);   /* Eco     */
  
}

void medir_distancia(){  
  digitalWrite(D6,LOW); /* Por cuestión de estabilización del sensor*/  
  delayMicroseconds(5);  
  digitalWrite(D6, HIGH); /* envío del pulso ultrasónico*/  
  delayMicroseconds(10);  
  tiempo=pulseIn(D5, HIGH);  
  distancia= int(0.017*tiempo); /*fórmula para calcular la distancia obteniendo un valor entero*/  
  /*Monitorización en centímetros por el monitor serial*/  
  Serial.print("Distancia :");  
  Serial.print(distancia);  
  Serial.println(" cm");  
} 
```

#### WIFI.hpp
No cambia con respecto a los descriptos mas arriba

```
void configuracion_wifi_STA()
{
  WiFi.begin(ssid, password);
  Serial.println(" ");
  Serial.println("Conectando");
  while (WiFi.status() != WL_CONNECTED)
  {
    delay(500);
    Serial.print(".");
  }
  Serial.println();
  Serial.print("Conectado, dirección IP: ");
  Serial.println(WiFi.localIP());
}

void configuracion_wifi_AP()
{
  WiFi.mode(WIFI_AP);  
  while(!WiFi.softAP(ssid, password))  
  {  
   Serial.println(".");  
    delay(100);  
  }  
  Serial.print("Iniciado AP ");  
  Serial.println(ssid);  
  Serial.print("IP address:\t");  
  Serial.println(WiFi.softAPIP());  
}
```

### server.hpp

```
void controlRaiz() {
  int i;
  Serial.println("Pagina original ");
  String s1 = Mi_pagina; //Read HTML contents
  String s3 = "<br> La medida es="+ String(distancia);
  String s=s1+s3+s2;
  server.send(200, "text/html", s); //Send web page
}

void controlLetrero() {
  Serial.println(server.arg("nombre"));
  String s1 = Mi_pagina; //Read HTML contents
  String s3 = "La medida es=";
  String s4 = String(distancia);
  
   String s=s1+s3+s4+s2;
  delay(200);
  server.send(200, "text/html", s); //Send web page
}
void iniciar_servidor()
{
  server.on("/", controlRaiz);
  server.begin();
}
```

#### archivo principal
Se cargan las librerias y se inicializa los dispositivos

```
#include "config.h"
#include <ESP8266WiFi.h>  
#include <ESP8266WebServer.h>
#include "sensor.hpp"
#include "actuador.hpp"
#include "WIFI.hpp"
#include "server.hpp"

void setup()  
{   
Serial.begin(115200);  
pinMode(LED_BUILTIN,OUTPUT);
Serial.println("Hola");
iniciarServo();  
iniciarUltrasonido();
configuracion_wifi_STA();  
iniciar_servidor();
}  
```
